In [1]:
import rasterio
import rasterio.plot
import numpy as np
from osgeo import gdal_array
from scipy.signal import savgol_filter
from scipy.interpolate import pchip_interpolate, InterpolatedUnivariateSpline
import math
from scipy.optimize import minimize
from scipy.optimize import Bounds
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
def double_logistic_array(params,timeArr):
    phenology=[]
    for time in timeArr:
        phenology.append(double_logistic(params,time))
    return np.array(phenology)

def double_logistic(params,time):
    """Generate a double logistic curve similar 
    to those of the MODIS phenology product
    
    k1   --- curvature parmeter for first half of season
    k2   --- curvature parmeter for second half of season
    t01  --- timing parmeter for first half of season
    t02  --- timing parmeter for second half of season
    c    --- minimum LAI/NDVI etc
    d    --- maximum LAI/NDVI etc
    
    time --- time valueto evaluate the function 
    """
    k1  =params[0]
    k2  =params[1]
    t01 =params[2]
    t02 =params[3]
    c   =params[4]
    d   =params[5]
    lgstc1=(c-d)/(1.+math.exp(k1*(time-t01)))+d    
    lgstc2=(d-c)/(1.+math.exp(k2*(time-t02)))+c
    return np.min([lgstc1,lgstc2])
    
def genSynthObs(params,freq=1,stddev=0.0):
    
    time_arr=[]
    synth_obs=[]
    for t in range(1,365,freq):
        time_arr.append(t)
        obs=double_logistic(params,t)
        synth_obs.append(obs)
        
    return np.array(synth_obs),np.array(time_arr)

def phenology_differential(synth_obs):
    differential = []
    for i in range(len(synth_obs) -1):
        differential.append(synth_obs[i+1] - synth_obs[i])
    sos = differential.index(max(differential)) +1
    eos = differential.index(min(differential)) +1
    return sos, eos

def phenology_optimize(params,sosBounds,eosBounds):
    sos = sos_optimize(params,sosBounds)
    eos = eos_optimize(params,eosBounds)
    return sos, eos

def sos_optimize(params,sosBounds):
    func = lambda x: 0 - double_logistic(params,x)
    grad_fun = np.grad()
    sos_opt = minimize(fun = func,
                       x0 = [params[2]],
                       bounds = sosBounds,
                       jac= '2-point',
                       options = {'ftol': 1e-1, 'gtol': 1e-6})
    return sos_opt.x
    
def eos_optimize(params,eosBounds):
    func = lambda x: double_logistic(params,x)
    eos_opt = minimize(fun = func,
                       x0 = [params[2]],
                       bounds = eosBounds,
                       jac= '2-point',
                       options = {'ftol': 1e-1, 'gtol': 1e-6})
    return eos_opt.x

def rmse(params,obs,obs_time):
    model=double_logistic_array(params,obs_time)
    rmse_series = np.sqrt(np.sum((model-obs)**2)/float(len(obs_time)))
    return rmse_series
    
def solver(params,obs,obs_time,costFunction,bounds):
    #costGradient, costHessian=ad.gh(costFunction)
    return minimize(costFunction,\
                    params,\
                    args=(obs,obs_time),\
                    bounds=bounds,\
                    #method='L-BFGS-B',\
                    jac='2-point',\
                    options={'gtol': 1e-6})

In [19]:
# 读取滤波后的影像
param_img_path = r'data/2019_params.tif'
param_img = gdal_array.LoadFile(param_img_path)
mask_img_path = r'data/smallregions.tif'
mask_img = gdal_array.LoadFile(mask_img_path)
out_sos_path = "data/" + str(year) + '_sos.tif'            
src = rasterio.open("data/" + str(year) + '.tif')
[doy, img_ht, img_wd] = param_img.shape              # 影像的维度（日期数、宽、高）
param_num = 6       # double logistic 参数个数
sosBounds = Bounds([70],[160])
eosBounds = Bounds([240],[340])

year = 2019


sos_img = np.zeros((1, img_ht, img_wd))
eos_img = np.zeros((1, img_ht, img_wd))

# 分配影像日期(DOY)
time_arr=np.arange(1, 365, 16)

# 开始计时
start = time.time()
for x in range(1,img_ht):
    # 打印进度信息，因为影像较大。
    if x%100 == 0:
        end = time.time()
        print(x, ' of ', img_ht, '\t', x/img_ht *100, '%\t Time elapsed: ', str(end - start) )
    for y in range(1,img_wd):
        # 如果序列全为 0，则不需要做拟合（研究区外）
        if mask_img[x,y] > 3:
            params = param_img[:,x,y] # 获取拟合参数
            sos_img[0,x,y], eos_img[0,x,y] = phenology_optimize(params,sosBounds,eosBounds)

out_sos_path = "data/" + str(year) + '_sos.tif'            
with rasterio.open(
    out_sos_path,
    'w',
    driver='GTiff',
    height=sos_img.shape[1],
    width=sos_img.shape[2],
    count=sos_img.shape[0],
    dtype=rasterio.float64, #存储数据精度为 float64,参数精度要求较高
    crs=src.crs,
    transform=src.transform,
) as dst:
    dst.write(sos_img)

out_eos_path = "data/" + str(year) + '_eos.tif'            
with rasterio.open(
    out_eos_path,
    'w',
    driver='GTiff',
    height=eos_img.shape[1],
    width=eos_img.shape[2],
    count=eos_img.shape[0],
    dtype=rasterio.float64, #存储数据精度为 float64,参数精度要求较高
    crs=src.crs,
    transform=src.transform,
) as dst:
    dst.write(eos_img)

100  of  6359 	 1.5725743041358704 %	 Time elapsed:  2.20001482963562
200  of  6359 	 3.145148608271741 %	 Time elapsed:  4.409754037857056
300  of  6359 	 4.717722912407611 %	 Time elapsed:  6.799982070922852
400  of  6359 	 6.290297216543482 %	 Time elapsed:  8.94795298576355
500  of  6359 	 7.862871520679351 %	 Time elapsed:  11.0985746383667
600  of  6359 	 9.435445824815222 %	 Time elapsed:  13.427573204040527
700  of  6359 	 11.008020128951093 %	 Time elapsed:  15.54899549484253
800  of  6359 	 12.580594433086963 %	 Time elapsed:  17.687122583389282
900  of  6359 	 14.153168737222835 %	 Time elapsed:  19.84124732017517
1000  of  6359 	 15.725743041358703 %	 Time elapsed:  22.003032207489014
1100  of  6359 	 17.298317345494574 %	 Time elapsed:  24.220882177352905
1200  of  6359 	 18.870891649630444 %	 Time elapsed:  26.799043655395508
1300  of  6359 	 20.443465953766314 %	 Time elapsed:  29.092029571533203
1400  of  6359 	 22.016040257902187 %	 Time elapsed:  31.225231409072876
15

In [17]:
year = 2019
out_sos_path = "data/" + str(year) + '_sos.tif'            
src = rasterio.open("data/" + str(year) + '.tif')
with rasterio.open(
    out_sos_path,
    'w',
    driver='GTiff',
    height=sos_img.shape[1],
    width=sos_img.shape[2],
    count=sos_img.shape[0],
    dtype=rasterio.float64, #存储数据精度为 float64,参数精度要求较高
    crs=src.crs,
    transform=src.transform,
) as dst:
    dst.write(sos_img)

out_eos_path = "data/" + str(year) + '_eos.tif'            
with rasterio.open(
    out_eos_path,
    'w',
    driver='GTiff',
    height=eos_img.shape[1],
    width=eos_img.shape[2],
    count=eos_img.shape[0],
    dtype=rasterio.float64, #存储数据精度为 float64,参数精度要求较高
    crs=src.crs,
    transform=src.transform,
) as dst:
    dst.write(eos_img)